In [1]:
import os
import shutil
import random

source_base_dir = '/kaggle/input/image-pro/Image_Processing'  


base_dir = '/kaggle/working/'


original_dir = os.path.join(source_base_dir, 'Original')
pixelated_dir = os.path.join(source_base_dir, 'Pixelated')


train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)


split_ratio = 0.8

def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)

    
    paired_files = [(file, file) for file in original_files if file in pixelated_files]

    random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]

    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
    
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))

split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio)

In [2]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Load train and test datasets
train_data = datasets.ImageFolder('/kaggle/working/train', transform=transform)
test_data = datasets.ImageFolder('/kaggle/working/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class ESPNet(nn.Module):
    def __init__(self):
        super(ESPNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: pixelated or original

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 64 * 64)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = ESPNet()


In [4]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:    # Print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')


[1,    10] loss: 1.596
[1,    20] loss: 0.482
[1,    30] loss: 0.433
[1,    40] loss: 0.460
[1,    50] loss: 0.438
[1,    60] loss: 0.411
[1,    70] loss: 0.379
[1,    80] loss: 0.350
[1,    90] loss: 0.332
[1,   100] loss: 0.338
[1,   110] loss: 0.370
[1,   120] loss: 0.340
[1,   130] loss: 0.292
[1,   140] loss: 0.372
[1,   150] loss: 0.295
[1,   160] loss: 0.343
[1,   170] loss: 0.385
[1,   180] loss: 0.337
[1,   190] loss: 0.347
[1,   200] loss: 0.374
[1,   210] loss: 0.362
[1,   220] loss: 0.371
[1,   230] loss: 0.358
[1,   240] loss: 0.331
[2,    10] loss: 0.312
[2,    20] loss: 0.335
[2,    30] loss: 0.277
[2,    40] loss: 0.389
[2,    50] loss: 0.339
[2,    60] loss: 0.357
[2,    70] loss: 0.365
[2,    80] loss: 0.349
[2,    90] loss: 0.319
[2,   100] loss: 0.311
[2,   110] loss: 0.284
[2,   120] loss: 0.314
[2,   130] loss: 0.336
[2,   140] loss: 0.325
[2,   150] loss: 0.373
[2,   160] loss: 0.368
[2,   170] loss: 0.321
[2,   180] loss: 0.324
[2,   190] loss: 0.287
[2,   200] 

In [5]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:    # Print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')


[1,    10] loss: 0.084
[1,    20] loss: 0.073
[1,    30] loss: 0.076
[1,    40] loss: 0.077
[1,    50] loss: 0.077
[1,    60] loss: 0.130
[1,    70] loss: 0.111
[1,    80] loss: 0.103
[1,    90] loss: 0.116
[1,   100] loss: 0.096
[1,   110] loss: 0.106
[1,   120] loss: 0.125
[1,   130] loss: 0.111
[1,   140] loss: 0.134
[1,   150] loss: 0.169
[1,   160] loss: 0.142
[1,   170] loss: 0.096
[1,   180] loss: 0.178
[1,   190] loss: 0.123
[1,   200] loss: 0.102
[1,   210] loss: 0.083
[1,   220] loss: 0.161
[1,   230] loss: 0.106
[1,   240] loss: 0.118
[2,    10] loss: 0.069
[2,    20] loss: 0.091
[2,    30] loss: 0.054
[2,    40] loss: 0.096
[2,    50] loss: 0.099
[2,    60] loss: 0.075
[2,    70] loss: 0.120
[2,    80] loss: 0.136
[2,    90] loss: 0.098
[2,   100] loss: 0.084
[2,   110] loss: 0.100
[2,   120] loss: 0.138
[2,   130] loss: 0.073
[2,   140] loss: 0.084
[2,   150] loss: 0.110
[2,   160] loss: 0.083
[2,   170] loss: 0.129
[2,   180] loss: 0.121
[2,   190] loss: 0.143
[2,   200] 

In [6]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test images: %d %%' % (
    100 * correct / total))


Accuracy on test images: 86 %


In [7]:
torch.save(model.state_dict(), '/kaggle/working/espNet_model.pth')
